In [102]:
import numpy as np
seed = 123
np.random.seed(seed)
import collections, copy, pickle
from termcolor import colored
from importlib import reload
import gc
from dateutil.parser import parse
import scipy.linalg, scipy.stats
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14
# rcParams['text.usetex'] = True
import seaborn as sns
from IPython.display import HTML

In [103]:
import sklearn.ensemble
import sklearn.svm
import sklearn.tree
import sklearn.linear_model
import sklearn.neighbors

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import sklearn.metrics

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [104]:
# Consider the training dataset
# Split the data up in n sets, based on `search_id` to prevent occurence of a `search_id` in multiple sets
#   CF score-matrices can be computed here as well
# Use crossvalidation on the n sets to select hyperparams
# Finally train model on full training dataset and make a prediction of the (unseen) test dataset

In [141]:
import util.data

In [142]:
data = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';', nrows=1*1000)
data_test = pd.read_csv('data/test_set_VU_DM_clean.csv', sep=';', nrows=1000)
# scores = pd.read_csv('data/scores_train.csv', sep=';')

In [143]:
ids = sklearn.utils.shuffle(data.srch_id.unique(), random_state=123)
N = ids.size
N # total number of samples

40

In [144]:
y_labels = [ k for k in data.columns if k not in data_test.columns ]
y_labels

['position', 'click_bool', 'booking_bool', 'score']

In [145]:
y_data = data[['click_bool', 'booking_bool', 'score']].copy()
x_data = data.drop(columns=y_labels)
# TODO split xy at the end

In [146]:
# clear memory
data = None
gc.collect();

In [147]:
n = 5 # number of cv splits
selection_size = np.floor(N/n).astype(int)
a = selection_size
# note that the final splice may be smaller
ids_selections = [ ids[i*a: min((i+1)*a, N)] for i in range(n) ]
assert len(ids_selections) == n
print(selection_size)

8


In [148]:
x_data_splits = [ x_data.loc[x_data.srch_id.isin(srch_ids)] for srch_ids in ids_selections ]
y_data_splits = [ y_data.iloc[x.index] for x in x_data_splits ]
# y_data_splits = [ y_data.loc[y_data.srch_id.isin(srch_ids)] for srch_ids in ids_selections ]
len(data_splits)

5

In [149]:
# we can now use (n-1) of the splits to train and the one other split to validate
# Now we will sample from a split to prevent class imbalance

In [156]:
# split slices again, into classes: select 1/3 booking, 1/3 click (but no booking), 1/3 none

In [151]:
x_train = x_data_splits[0]
y_train = y_data_splits[0]
assert x_train.shape[0] == y_train.shape[0]

In [168]:
# index = x_train.index
y_bookings = y_train.query('booking_bool == 1')
y_clicks = y_train.query('click_bool == 1 and booking_bool != 0')
y_others = y_train.query('click_bool != 1')

x_bookings = x_train.loc[y_bookings.index]
x_clicks = x_train.loc[y_clicks.index]
x_others = x_train.loc[y_others.index]

n_max = max([y.shape[0] for y in [y_bookings, y_clicks, y_others] ])
n_min = min([y.shape[0] for y in [y_bookings, y_clicks, y_others] ])
n_max, n_min

(203, 5)

In [164]:
# sample splitted splices
index = sklearn.utils.resample(y_bookings.index, replace=True, random_state=123)
y_bookings.loc[index]
# note the duplicate entries

,click_bool,booking_bool,score
543,1,1,5
944,1,1,5
543,1,1,5
243,1,1,5
899,1,1,5


In [169]:
# TODO sample n_max items during training epochs

sklearn.utils.resample(y_bookings, y_clicks, y_others, replace=True, random_state=123)

ValueError: Found input variables with inconsistent numbers of samples: [5, 5, 203]

In [ ]:
model = sklearn.ensemble.RandomForestClassifier(n_estimators=100)
scores_acc = cross_val_score(model, x_train, y_train, cv=n_folds, scoring='accuracy') # roc_auc accuracy